#### This Code caluclates the Elevation_Difference Uncertainty based on the Standard Error Approach from Gardelle et al. 2013
### For further reading, please refer Gardelle et al. 2013 (doi:10.5194/tc-7-1263-2013) or Goerlich et al. 2017 (doi:10.3390/rs9030275)

In [1]:
#importing the libraries required for the evaluation
import numpy as np
import rasterio
from pygeotools.lib import malib, warplib, iolib, geolib
import numpy.ma as ma
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

#### Creating two seperate dataframes for glacier and bedrock areas 
##### Calculating stats for both the cases
##### Joining both dataframes based on the index at Rounded Elevation value

In [2]:
#dataframe (df) for glacier pixels
fn1='B-tile-0.tif' #Elevation_dataset, it is assumed that both the elevation and dh rasters are clipped to glacier extents
fn2='WV_CArto-tile-0-last_shpclip.tif' #Elevation_difference dataset
ds_list=warplib.memwarp_multi_fn([fn1,fn2],res='max',extent='intersection',t_srs='first', r='bilinear')
A=iolib.ds_getma(ds_list[0])
dh=iolib.ds_getma(ds_list[1])
E=ma.asarray(A,dtype=np.float32)
common_mask=malib.common_mask([E,dh])
dh_glacier=ma.compressed(ma.array(dh,mask=common_mask))
DEM_glacier=ma.compressed(ma.array(E,mask=common_mask))
raw={'Elevation':DEM_glacier,
    'dh':dh_glacier}
df=pd.DataFrame(raw, columns=['Elevation','dh'])
#dataframe (df1) for bedrock (ice-free stable_Areas)
fn3='B-tile-0.tif' #Elevation_dataset, it is assumed that both the elevation and dh rasters are clipped to glacier extents
fn4='WV_CArto-tile-0-last_masked.tif' #Elevation_difference dataset
ds_list_1=warplib.memwarp_multi_fn([fn3,fn4],res='max',extent='intersection',t_srs='first', r='bilinear')
B=iolib.ds_getma(ds_list_1[0])
dh1=iolib.ds_getma(ds_list_1[1])
E1=ma.asarray(B,dtype=np.float32)
c_mask=malib.common_mask([dh1,E1])
dh_stable=ma.compressed(ma.array(dh1,mask=c_mask))
DEM_stable=ma.compressed(ma.array(E1,mask=c_mask))
raw_stable={'Elevation':DEM_stable,
    'dh':dh_stable}
df2=pd.DataFrame(raw_stable, columns=['Elevation','dh'])
#Rounding Glacier Elevation to next nearest multiple of 50
df['Elevation_rounded']=(df['Elevation']+49)//50*50
#Calulating_stats_for_Glacier_Pixels
grouped=df.groupby('Elevation_rounded')
df1=grouped['dh'].agg([np.count_nonzero,np.sum, np.mean, np.std])
#Rounding Bedrock Elevation to next nearest multiple of 50
df2['Elevation_rounded']=(df2['Elevation']+49)//50*50
#Calulating_stats_for_Bedrock_Pixels
grouped_stable=df2.groupby('Elevation_rounded')
df3=grouped_stable['dh'].agg([np.count_nonzero,np.sum, np.mean, np.std])
#Joining the stats dataframes (df1*Glacier* and df3*Stable*) into a common"df4"
df4=df3.join(df1,how='left', lsuffix="Stable", rsuffix="Glacier", sort=False)
df4


Warping all inputs to the following:
Resolution: 10.0
Extent: [551565.0, 3029605.0, 704093.339, 3115655.0]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: B-tile-0.tif
nl: 8605 ns: 15253 res: 10.000
2 of 2: WV_CArto-tile-0-last_shpclip.tif
nl: 8605 ns: 15253 res: 10.000

Warping all inputs to the following:
Resolution: 10.0
Extent: [551565.0, 3029605.0, 704093.339, 3115655.0]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: B-tile-0.tif
nl: 8605 ns: 15253 res: 10.000
2 of 2: WV_CArto-tile-0-last_masked.tif
nl: 8605 ns: 15253 res: 10.000


,count_nonzeroStable,sumStable,meanStable,stdStable,count_nonzeroGlacier,sumGlacier,meanGlacier,stdGlacier
Elevation_rounded,,,,,,,,
1950.0,14.0,-6.380981,-0.455784,11.346849,NaN,NaN,NaN,NaN
2000.0,104.0,-38.507202,-0.370262,20.899368,NaN,NaN,NaN,NaN
2050.0,162.0,1168.069946,7.210308,13.876693,NaN,NaN,NaN,NaN
2100.0,192.0,-800.042725,-4.166889,13.446272,NaN,NaN,NaN,NaN
2150.0,208.0,-1468.926025,-7.062144,14.679708,NaN,NaN,NaN,NaN
2200.0,107.0,-2095.551270,-19.584591,15.636267,NaN,NaN,NaN,NaN
2250.0,97.0,-1531.335205,-15.786961,12.949129,NaN,NaN,NaN,NaN
2300.0,6.0,-131.814941,-21.969156,2.800151,NaN,NaN,NaN,NaN
2700.0,12.0,80.741211,6.728434,4.925403,NaN,NaN,NaN,NaN


In [3]:
#Effective_No_of_Pixels "Equation 5 in Goerlich et al. 2017"
df4['Effective_stable_pixels']=(df4['count_nonzeroStable']*10)/(2*72.9)
distance_of_auto_correlation=72.9 #Calculated from semivariogram for the bedrock dh values, working on the script of semivariogram
df4['SE']=(df4['stdStable'])/(np.sqrt(df4['Effective_stable_pixels']))#Standard Error for each interval "Equation 4 in Goerlich et al. 2017"

In [4]:
#Weighing the Standard Error in each interval with glacier hypsometry of the corresponding interval
df4['SE*Count_Glacier']=df4['SE']*df4['count_nonzeroGlacier']
SUM_SE_Count=df4['SE*Count_Glacier'].sum()
Sum_Count=df4['count_nonzeroGlacier'].sum()
Elevation_difference_error=(SUM_SE_Count)/(Sum_Count)
print'Error_in_Elevation_difference_estimate is %s m' %Elevation_difference_error

Error_in_Elevation_difference_estimate is 0.155473 m


In [5]:
df4

,count_nonzeroStable,sumStable,meanStable,stdStable,count_nonzeroGlacier,sumGlacier,meanGlacier,stdGlacier,Effective_stable_pixels,SE,SE*Count_Glacier
Elevation_rounded,,,,,,,,,,,
1950.0,14.0,-6.380981,-0.455784,11.346849,NaN,NaN,NaN,NaN,0.960219,11.579506,NaN
2000.0,104.0,-38.507202,-0.370262,20.899368,NaN,NaN,NaN,NaN,7.133059,7.825196,NaN
2050.0,162.0,1168.069946,7.210308,13.876693,NaN,NaN,NaN,NaN,11.111111,4.163008,NaN
2100.0,192.0,-800.042725,-4.166889,13.446272,NaN,NaN,NaN,NaN,13.168724,3.705357,NaN
2150.0,208.0,-1468.926025,-7.062144,14.679708,NaN,NaN,NaN,NaN,14.266118,3.886552,NaN
2200.0,107.0,-2095.551270,-19.584591,15.636267,NaN,NaN,NaN,NaN,7.338820,5.771915,NaN
2250.0,97.0,-1531.335205,-15.786961,12.949129,NaN,NaN,NaN,NaN,6.652949,5.020344,NaN
2300.0,6.0,-131.814941,-21.969156,2.800151,NaN,NaN,NaN,NaN,0.411523,4.365003,NaN
2700.0,12.0,80.741211,6.728434,4.925403,NaN,NaN,NaN,NaN,0.823045,5.429126,NaN
